<span style=" color: yellow; font-size: 24px;">Deep Learning 1 Project _ Group 5:<br>Tweet Sentiment Extraction (Kaggle · Featured Code Competition)<br></span><span style="font-size: 22px;">
Shrey	Patel	101541370<br>
Sam	Emami	101575471<br>
Eric	Lessa	101549935<br>
Dwip	Makwana	101483523<br>
Moossa	Hussain	101542820<br>
Chaoyu	Liu	101573622<br>
Devanshi 	Dave	101582208<br>
Rutika	Bhuva	101551781<br>
</span>

<span style=" color: yellow; font-size: 24px;">Use This Template **by memeber name**: </span>description for each part<span style="font-size: 22px;">

In [8]:
!pip install shap
!pip install transformers
!pip install pipeline
!pip install datasets
!pip show torchvision
!pip install evaluate

#go to https://graphviz.gitlab.io/download/ and install graphviz

Name: torchvision
Version: 0.20.1+cu121
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, pillow, torch
Required-by: easyocr, fastai, timm
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00


<span style="color: yellow; font-size: 24px;">Hugging Face Transformers Tools for Using Pre-trained LLMs:<br><br></span>
<span style="color: yellow; font-size: 24px;">pipeline: </span><span style="font-size: 22px;"> It is like a convenient wrapper or helper that simplifies the process of working with models such as those loaded with AutoModelForSequenceClassification and their associated tokenizers.<br>It integrates with pre-trained models and tokenizers for specific tasks and perform NLP tasks and the processes such as:<br>
Loading the model.<br>
Tokenizing the input text.<br>
Generating predictions.<br>
Post-processing the output (if necessary)<br><br></span>
<span style="color: yellow; font-size: 24px;">AutoModelForSequenceClassification: </span><span style="font-size: 22px;"> Loading Pre-trained Models designed for sequence classification tasks. Sequence Classification refers to tasks where you classify an entire sequence of text (like a sentence or document) into a category. Examples include:<br>
<span style="color: red">Sentiment analysis (positive, negative, neutral)</span><br>
Topic classification (news, sports, politics)<br>
Spam detection<br>
Intent classification (e.g., order, question, complaint)<br>
"Auto" in the Name: The Auto prefix in the class name signifies its flexibility. You can use it to load various pre-trained models from different architectures (BERT, RoBERTa, DistilBERT, etc.) simply by specifying the model name or path.<br><br></span>
<span style="color: yellow; font-size: 24px;">AutoTokenizer: </span><span style="font-size: 22px;"> tokenizing the inputs. The tokenizer ensures that:<br>
The tokenized input matches the vocabulary and format expected by the model.<br>
The inputs include features like input_ids, attention_mask, and token_type_ids (if applicable).<br><br></span>

In [9]:
import os
import pandas as pd
import shap
import numpy as np
import torch
from datasets import Dataset
import evaluate
from transformers import EarlyStoppingCallback
from transformers import EvalPrediction
from transformers import (
    pipeline,
    AutoModelForSequenceClassification,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    pipeline,
    DefaultDataCollator,
    DataCollatorWithPadding,
    AutoTokenizer
)

In [10]:
# Disable wandb
os.environ["WANDB_MODE"] = "disabled"

In [52]:
class SentimentExtraction:
    
    def __init__(self, train_path):
        self.train_df = pd.read_csv(train_path)
        self.tokenizer = None
        self.model = None
        self.squad_metric = evaluate.load("squad")

    def prepare_data(self):
        """Clean and prepare the data"""
        print("Dataset columns:", self.train_df.columns.tolist())
        print("\nSample data:")
        print(self.train_df.head())
        
        self.train_df = self.train_df.dropna(subset=['text'])
        
        dataset = Dataset.from_pandas(self.train_df)
        split_data = dataset.train_test_split(test_size=0.1, seed=42)
        return split_data['train'], split_data['test']

    def setup_model(self, model_name):
        """Initialize the model and tokenizer"""
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        # Initialize for binary classification (2 labels)
        self.model = AutoModelForQuestionAnswering.from_pretrained(model_name)
        
        # Move model to available device
        device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model.to(device)

    def prepare_train_features(self, examples):
        sentiment = examples['sentiment']
        tweet = examples['text']
        span = examples['selected_text']

        tokenized_qa = self.tokenizer(sentiment, # question
                                tweet, # context
                                padding='max_length',
                                return_offsets_mapping=True)

        qa_tokens = tokenized_qa["input_ids"]

        span_tokens = self.tokenizer(span)["input_ids"]

        tokenized_qa["start_positions"] = []
        tokenized_qa["end_positions"] = []

        start_char = tweet.find(span)
        end_char = start_char + len(span)

        offsets = tokenized_qa.pop("offset_mapping")
        start_token = end_token = None
        for idx, (start, end) in enumerate(offsets):
            if start <= start_char < end:
                start_token = idx
            if start < end_char <= end:
                end_token = idx
                break

        tokenized_qa["start_positions"].append(start_token)
        tokenized_qa["end_positions"].append(end_token)

        if start is None or end is None:
            print(tweet + ' -> ' + span)
            print(f' {qa_tokens} -> {span_tokens}')
            return None

        return tokenized_qa

    def compute_metrics(self, eval_pred):
        predictions, labels = eval_pred
        start_preds, end_preds = predictions  # Unpack start and end logits
        start_labels, end_labels = labels     # Unpack start and end positions

        # Convert logits to predicted indices
        start_preds = np.argmax(start_preds, axis=1)  # Shape: (100,)
        end_preds = np.argmax(end_preds, axis=1)      # Shape: (100,)

        total_jaccard = 0
        num_samples = len(start_preds)

        # Create prediction and reference text spans
        preds = []
        refs = []
        for i in range(len(start_preds)):
            # Decode predicted text span
            pred_tokens = self.tokenizer.decode(range(start_preds[i], end_preds[i] + 1))            
            preds.append({"id": str(i), "prediction_text": pred_tokens})
            
            # Decode reference text span
            ref_tokens = self.tokenizer.decode(range(start_labels[i, 0], end_labels[i, 0] + 1))
            refs.append({"id": str(i), "answers": {"text": [ref_tokens], "answer_start": [start_labels[i, 0]]}})

            pred_set = set(pred_tokens.lower().split())
            ref_set = set(ref_tokens.lower().split())
            intersection = pred_set.intersection(ref_set)
            union = pred_set.union(ref_set)
            jaccard = len(intersection) / len(union) if union else 0
            total_jaccard += jaccard

        avg_jaccard = total_jaccard / num_samples

        # Compute SQuAD metrics
        squad_result = self.squad_metric.compute(predictions=preds, references=refs)

        return {
            "exact_match": f"{squad_result['exact_match']:.2f}%",
            "f1_score": f"{squad_result['f1']:.2f}%",
            "avg_jaccard": f"{avg_jaccard:.2f}"
        }

    def train(self, model_name, checkpoint=None):
        # Prepare data
        print("\nPreparing datasets...")
        train_dataset, eval_dataset = self.prepare_data()
        print(f"Train dataset size: {len(train_dataset)}")
        print(f"Eval dataset size: {len(eval_dataset)}")

        # Setup model
        print("\nSetting up model...")
        self.setup_model(model_name)

        tokenized_train = train_dataset.map(self.prepare_train_features)

        tokenized_eval = eval_dataset.map(self.prepare_train_features)

        print(f"\nSelecting {tokenized_train} training samples and {tokenized_eval} evaluation samples...")

        args = TrainingArguments(
            f"finetune-BERT-tweet",
            evaluation_strategy = "epoch",
            learning_rate=2e-5,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            num_train_epochs=3,
            weight_decay=0.01,
            save_total_limit=2,
            fp16=True
        )

        #early_stopping = EarlyStoppingCallback(early_stopping_patience=3)  # Stop if no improvement after 3 evaluations

        #small_train_dataset = tokenized_train.shuffle(seed=42).select(range(500))
        #small_eval_dataset = tokenized_eval.shuffle(seed=42).select(range(100))

        trainer = Trainer(
            model=self.model,
            args=args,
            train_dataset=tokenized_train,
            eval_dataset=tokenized_eval,
            #train_dataset=small_train_dataset,
            #eval_dataset=small_eval_dataset,
            data_collator=DataCollatorWithPadding(self.tokenizer),
            tokenizer=self.tokenizer,
            compute_metrics=self.compute_metrics
         #    callbacks=[early_stopping]
        )

        # Train model
        print("\nStarting training...")
        try:
            trainer.train(resume_from_checkpoint=checkpoint)
            print("\nTraining completed successfully!")
        except Exception as e:
            print(f"\nError during training: {str(e)}")
            return
        
        # Save the model
        print("\nSaving model...")
        trainer.save_model("sentiment_extraction_tweet_model")

    def baseline(self):

        dataset, _ = self.prepare_data()

        total_jaccard = 0
        preds = []
        refs = []
        for i in range(len(dataset)):
            sample = dataset[i]
            sentiment = sample['sentiment']
            tweet = sample['text']
            span = sample['selected_text']
    
            inputs = self.tokenizer(
                sentiment,  # the sentiment question
                tweet,      # the tweet context
                padding=True,
                truncation=True,
                return_tensors="pt"
            )
        
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
            inputs = {k: v.to(device) for k, v in inputs.items()}
           
            with torch.no_grad():
                outputs = self.model(**inputs)
                start_logits = outputs.start_logits
                end_logits = outputs.end_logits
        
                start_position = torch.argmax(start_logits, dim=-1)
                end_position = torch.argmax(end_logits, dim=-1)
        
                tokens = self.tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
                
                pred_span_tokens = tokens[start_position:end_position+1]
                pred_text = self.tokenizer.convert_tokens_to_string(pred_span_tokens)

                preds.append({"id": str(i), "prediction_text": pred_text})
                refs.append({"id": str(i), "answers": {"text": [span], "answer_start": [tweet.find(span)]}})

                set1 = set(pred_text.lower().split())
                set2 = set(span.lower().split())
                jaccard = len(set1 & set2) / len(set1 | set2)  # Intersection / Union
                total_jaccard += jaccard
        
        squad_result = self.squad_metric.compute(predictions=preds, references=refs)
        print("Prediction results without fine-tuning:")
        print(f"\texact_match: {squad_result['exact_match']:.2f}%")
        print(f"\tf1_score: {squad_result['f1']:.2f}%")
        print(f"\tavg_jaccard: {total_jaccard/len(dataset):.2f}")


In [46]:
def main():
    # Initialize processor
    print("Initializing data processor...")
    processor = SentimentExtraction('/kaggle/input/tweet-extraction/train.csv')

    #model_name = "bert-base-uncased"
    model_name = "distilbert-base-uncased"
    processor.train(model_name)

    return processor

def prediction_test(processor):
    tweet = " you guys didn`t say hi or answer my questions yesterday  but nice songs."
    sentiment = "positive"

    inputs = processor.tokenizer(
        sentiment,  # the sentiment question
        tweet,      # the tweet context
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = processor.model(**inputs)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

        start_position = torch.argmax(start_logits, dim=-1)
        end_position = torch.argmax(end_logits, dim=-1)

        tokens = processor.tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
        
        sentiment_span = tokens[start_position:end_position+1]
        sentiment_text = processor.tokenizer.convert_tokens_to_string(sentiment_span)
        
        print(f"Predicted sentiment span: {sentiment_text}")

def print_baseline():
    # Initialize processor
    print("Initializing data processor...")
    processor = SentimentExtraction('/kaggle/input/tweet-extraction/train.csv')

    #model_name = "bert-base-uncased"
    model_name = "distilbert-base-uncased"
    processor.setup_model(model_name)
    processor.baseline()



In [53]:
print_baseline()

Initializing data processor...


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Dataset columns: ['textID', 'text', 'selected_text', 'sentiment']

Sample data:
       textID                                               text  \
0  cb774db0d1                I`d have responded, if I were going   
1  549e992a42      Sooo SAD I will miss you here in San Diego!!!   
2  088c60f138                          my boss is bullying me...   
3  9642c003ef                     what interview! leave me alone   
4  358bd9e861   Sons of ****, why couldn`t they put them on t...   

                         selected_text sentiment  
0  I`d have responded, if I were going   neutral  
1                             Sooo SAD  negative  
2                          bullying me  negative  
3                       leave me alone  negative  
4                        Sons of ****,  negative  
Prediction results without fine-tuning:
	exact_match: 0.37%
	f1_score: 1.70%
	avg_jaccard: 0.01


In [54]:
processor = main()

Initializing data processor...

Preparing datasets...
Dataset columns: ['textID', 'text', 'selected_text', 'sentiment']

Sample data:
       textID                                               text  \
0  cb774db0d1                I`d have responded, if I were going   
1  549e992a42      Sooo SAD I will miss you here in San Diego!!!   
2  088c60f138                          my boss is bullying me...   
3  9642c003ef                     what interview! leave me alone   
4  358bd9e861   Sons of ****, why couldn`t they put them on t...   

                         selected_text sentiment  
0  I`d have responded, if I were going   neutral  
1                             Sooo SAD  negative  
2                          bullying me  negative  
3                       leave me alone  negative  
4                        Sons of ****,  negative  
Train dataset size: 24732
Eval dataset size: 2748

Setting up model...


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/24732 [00:00<?, ? examples/s]

Map:   0%|          | 0/2748 [00:00<?, ? examples/s]


Selecting Dataset({
    features: ['textID', 'text', 'selected_text', 'sentiment', '__index_level_0__', 'input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 24732
}) training samples and Dataset({
    features: ['textID', 'text', 'selected_text', 'sentiment', '__index_level_0__', 'input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 2748
}) evaluation samples...

Starting training...


`evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
`tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.


Epoch,Training Loss,Validation Loss,Exact Match,F1 Score,Avg Jaccard
1,No log,2.805752,9.00%,44.81%,0.37
2,No log,2.130236,29.00%,59.22%,0.52
3,No log,1.948870,34.00%,60.46%,0.54



Training completed successfully!

Saving model...


In [55]:
prediction_test(processor)

Predicted sentiment span: you guys didn ` t say hi or answer my questions yesterday but nice songs.


In [ ]:
def trainFromCheckpoint(checkpoint):
    # Initialize processor
    print("Initializing data processor...")
    processor = SentimentExtraction('/kaggle/input/tweet-extraction/train.csv')

    processor.train(checkpoint)


trainFromCheckpoint('/kaggle/working/finetune-BERT-tweet/checkpoint-8000')

<span style="font-size: 22px;">From the Hugging Face Transformers library, acquires a pre-trained BERT-based sentiment analysis model (**LLM**), with its appropriate pipeline (**classifier**) and **tokenizer**.</span>


### Predictions sentiment extraction

<span style=" color: yellow; font-size: 24px;">Importan Note by Sam: </span><span style="font-size: 22px;">LLMs such as BERT, LLama, GPTT,etc., and BERT-based models (like distilbert-base-uncased-finetuned-sst-2-english) do not include an activation function (e.g., softmax or sigmoid) at their output layer by default. Instead, they output logits, which are raw, unnormalized scores for each class. The activation function, if needed, is applied externally depending on the specific task and requirements.<br> BERT is a general-purpose transformer model. By omitting the activation function, it can be fine-tuned for various tasks: Multi-class classification (e.g., softmax), Binary classification (e.g., sigmoid), Regression tasks (no activation function).<br></span>


<span style=" color: yellow; font-size: 24px;">outputs</span> <span style="font-size: 22px;">It is an object of type ModelOutput, specifically tailored for the model we are using. Here we didn't simply print the result of the sentiment for the each tweet, but we used output.logits to go access the detailed classification results for each text for further word by word analysis in SHAP.<br></span>
<span style=" color: yellow; font-size: 24px;">logits:</span> <span style="font-size: 22px;"> is the model’s raw output before applying the activation function. logits are unnormalized scores, which can be positive, negative, or zero, depending on the model’s confidence in each class. Here "outputs.logits" is a tensor containing the raw, unnormalized predictions for each input sequence. Each row in the tensor corresponds to one input in texts, and each column corresponds to a class.<br></span>
<span style=" color: yellow; font-size: 24px;">Normalize logits to probabilities:</span><span style="font-size: 22px;"> by applying softmax we normalize the raw predictions for each tweet to probabilities (the results are printed for clarity).<br></span>
<span style=" color: yellow; font-size: 24px;">detach()</span><span style="font-size: 22px;"> We use it since we don’t want to compute gradients for the tensor. Since probabilities are used only for prediction (not training), there’s no need to track gradients, so detach() avoids unnecessary computational overhead.<br></span>


In [ ]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
# Load pre-trained sentiment analysis model and tokenizer
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
classifier = pipeline("sentiment-analysis", model=model_name)

# Tokenizer and model for SHAP
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Define a wrapper for the classifier to work with SHAP
def classifier_wrapper(texts):
    tokens = tokenizer(list(texts), padding=True, truncation=True, return_tensors="pt")
    outputs = model(**tokens)
    probabilities = outputs.logits.softmax(dim=1).detach().numpy()
    print(f"\nlogits contains tweets' unnormalized prediction scores for each class(positive / negative):\n{outputs.logits}\n")
    return probabilities

# Select a subset of the training data for SHAP analysis
sample_texts = train_df["text"].sample(10, random_state=42).tolist()
print("Sample Texts:")
print(sample_texts)
print(f"Probibilities after normalizing logits by softwax:\n {classifier_wrapper(sample_texts)}")  # Should output a numpy array of probabilities


<span style=" color: yellow; font-size: 24px;">SHAP Analysis:</span><span style="font-size: 22px;"> SHAP is based on <u>Shapley values</u> which is a concept from <u>Game Theory</u>. Imagine a group of players collaborating to achieve a common goal. Shapley values determine how to fairly distribute the rewards or profits among the players based on their individual contributions.
In the context of machine learning, features of a data point can be seen as "players" contributing to the model's prediction.<br> SHAP aims to fairly attribute the "credit" (or "blame") for the model's output to each feature.<br>
For complex models, SHAP often uses <u>approximations</u> or <u>sampling techniques</u> to estimate Shapley values.<br>
The explainer considers <u>all possible combinations of features (words or tokens)</u> in the sample.
For each combination, it calculates the model's prediction.<br>
This process helps determine how much each feature contributes to the overall prediction.
Shapley Values are assigned based on the calculated predictions for different feature combinations,
SHAP assigns a value to each feature.
</br></br></spn>
</span>
<span style=" color: yellow; font-size: 24px;">shap.Explainer:</span><span style="font-size: 22px;"> Is the core class in the SHAP library. It encapsulates the model and the method for <u>calculating</u> SHAP values, which quantify the <u>contribution of each feature</u> (in this case, words or tokens) <u>to the model's prediction</u> for a given sample.<br>
The Explainer needs "classifier_wrapper" function to understand how the model makes predictions.<br><br></span>

<span style=" color: yellow; font-size: 24px;">Note: </span><span style="font-size: 22px;">
When classifier_wrapper is run by SHAP, there are six or less probibilities for the ten tweets in sample_texts. It is because the tokenizer applies padding and truncation to the input. If multiple texts are identical, the tokenizer may return the same tokens for duplicates. If sample_texts contains duplicate or very similar tweets, their logits will be identical, and we might get fewer output tensors.<br><br></span>

In [ ]:

# Explain predictions using SHAP
# This line integrates the model in the function to SHAP and encapsulates
# the necessary information to compute SHAP values for the given model and data.
explainer = shap.Explainer(classifier_wrapper, tokenizer)
shap_values = explainer(sample_texts)

# Visualize the SHAP values for each word:
# Output 0 and Output 1: predicted probabilities for two classes
for i, text in enumerate(sample_texts):
    print(f"\nSHAP Analysis for Text {i + 1}: {text}")
    shap.plots.text(shap_values[i])
'''
#The following code save SHAP analysis results into different html files for each tweet:

#shap.save_html("shap_analysis.html", shap_values)
# Manually set the base value if explainer.expected_value is None
# For classification tasks, the base value can be the mean of logits
if explainer.expected_value is None:
    print("explainer.expected_value is None. Computing a manual base value.")
    base_value = np.mean([np.mean(values.values) for values in shap_values])
else:
    base_value = explainer.expected_value

# Loop through all SHAP values and save each as a separate force plot
for i, shap_value in enumerate(shap_values):
    # Debugging: Print base value and SHAP values
    print(f"Processing Text {i + 1}")
    print(f"Base Value: {base_value}")
    print(f"SHAP Values for Text {i + 1}: {shap_value.values}")

    # Check if shap_value.values is None
    if shap_value.values is None:
        print(f"Skipping Text {i + 1} due to missing values.")
        continue

    # Handle multi-output models (if applicable)
    if isinstance(base_value, list):
        base_value = base_value[0]  # Use the base value for the first output

    # Create a force plot for the current sample
    try:
        vis = shap.plots.force(base_value, shap_value.values)

        # Save the visualization as an HTML file
        file_name = f"shap_analysis_{i + 1}.html"
        shap.save_html(file_name, vis)
        print(f"Saved SHAP visualization for Text {i + 1} as {file_name}")
    except Exception as e:
        print(f"Error processing Text {i + 1}: {e}")
'''

In [ ]:
!zip -r checkpoint-9276.zip /kaggle/working/finetune-BERT-tweet/checkpoint-9276